In [1]:
import pandas as pd
import numpy as np
from sklearn.utils import class_weight
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow import keras
from tensorflow.keras.models import Sequential, load_model
import json
from joblib import dump

In [2]:
df = pd.read_csv("./csv/combined_mbti_df.csv")

df = df.drop_duplicates().dropna()

df.tail(5)

,mbti,function_pair,danceability_mean,danceability_stdev,energy_mean,energy_stdev,loudness_mean,loudness_stdev,mode_mean,mode_stdev,...,Aminor_count,AMajor_count,A#/Bbminor_count,BMajor_count,Dminor_count,D#_Ebminor_count,Gminor_count,A#/BbMajor_count,F#/GbMajor_count,Bminor_count
4076,ESTJ,SJ,0.552889,0.155100,0.595611,0.137195,-7.224889,2.101033,0.500000,0.514496,...,2.0,0.0,0.0,1.0,0.0,0.0,2.0,1.0,0.0,2.0
4077,ESTJ,SJ,0.517780,0.142557,0.674940,0.182267,-7.902200,2.822676,0.600000,0.494872,...,3.0,3.0,0.0,5.0,1.0,1.0,2.0,2.0,1.0,4.0
4078,ESTJ,SJ,0.585313,0.181908,0.694375,0.173636,-5.307063,1.531874,0.562500,0.512348,...,1.0,1.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4079,ESTJ,SJ,0.636040,0.152382,0.652420,0.176042,-6.553020,2.813042,0.540000,0.503457,...,2.0,0.0,3.0,1.0,0.0,1.0,2.0,2.0,1.0,3.0
4080,ESTJ,SJ,0.640733,0.145205,0.678333,0.182482,-5.763733,2.075266,0.555556,0.502519,...,2.0,0.0,4.0,1.0,2.0,0.0,3.0,1.0,2.0,2.0


In [26]:
playlist_features = [
    "danceability_mean",
    "energy_mean",
    "mode_mean",
    "speechiness_mean",
    "liveness_mean",
    "valence_mean",
    "tempo_mean",
    "instrumentalness_mean",
]

### CONVERT TO 4 LETTER COLUMNS

In [29]:
df["IE"] = df["mbti"].str[0].map({'I': 0, 'E': 1})
df["NS"] = df["mbti"].str[1].map({'N': 0, 'S': 1})
df["TF"] = df["mbti"].str[2].map({'T': 0, 'F': 1})
df["JP"] = df["mbti"].str[3].map({'J': 0, 'P': 1})

df.head(5)

,mbti,function_pair,danceability_mean,danceability_stdev,energy_mean,energy_stdev,loudness_mean,loudness_stdev,mode_mean,mode_stdev,...,Dminor_count,D#_Ebminor_count,Gminor_count,A#/BbMajor_count,F#/GbMajor_count,Bminor_count,IE,NS,TF,JP
0,INFP,NF,0.557841,0.155011,0.553325,0.225178,-8.352591,3.273317,0.659091,0.479495,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,1
1,INFP,NF,0.587636,0.135644,0.556273,0.191642,-8.215697,3.356867,0.636364,0.488504,...,1.0,1.0,1.0,1.0,0.0,0.0,0,0,1,1
2,INFP,NF,0.677000,0.128009,0.851280,0.128336,-5.046100,2.180554,0.580000,0.498569,...,1.0,0.0,0.0,1.0,2.0,9.0,0,0,1,1
3,INFP,NF,0.517000,0.169477,0.513412,0.258345,-10.172833,4.935140,0.785714,0.415300,...,0.0,1.0,0.0,3.0,0.0,0.0,0,0,1,1
4,INFP,NF,0.560400,0.141450,0.445862,0.242592,-10.572240,5.685179,0.820000,0.388088,...,0.0,0.0,0.0,5.0,1.0,1.0,0,0,1,1


In [30]:
x = df[playlist_features]

scaler = StandardScaler()
x = scaler.fit_transform(x)
dump(scaler, "models/mbti_scaler.pkl") 

['models/mbti_scaler.pkl']

### TRAIN MODEL (BY EACH LETTER)

In [31]:
def train_model(x, y):
    x_train, x_test, y_train, y_test = train_test_split(
        x, y, 
        test_size=0.3, 
        random_state=42, 
        shuffle=True)

    model = Sequential([
        keras.layers.Dense(128, activation='relu', input_dim=x.shape[1]),
        keras.layers.Dropout(0.2),
        
        keras.layers.Dense(64, activation='relu'),
        keras.layers.Dropout(0.2),

        keras.layers.Dense(32, activation='relu'),
        keras.layers.Dropout(0.1),

        keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy'],
    )

    model.fit(x_train, y_train,
              epochs = 30, 
              validation_split=0.1)
    
    loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
    
    return model, accuracy, x_test, y_test


In [32]:
ie_model, ie_accuracy, x_ie_test, y_ie_test = train_model(x, df["IE"])

Epoch 1/30


C:\Users\DELL\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


79/79 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.6115 - loss: 0.6507 - val_accuracy: 0.7214 - val_loss: 0.6118
Epoch 2/30
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6949 - loss: 0.5981 - val_accuracy: 0.7071 - val_loss: 0.6071
Epoch 3/30
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6981 - loss: 0.5942 - val_accuracy: 0.6964 - val_loss: 0.6045
Epoch 4/30
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7176 - loss: 0.5730 - val_accuracy: 0.7071 - val_loss: 0.5960
Epoch 5/30
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7173 - loss: 0.5759 - val_accuracy: 0.7036 - val_loss: 0.5992
Epoch 6/30
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7093 - loss: 0.5878 - val_accuracy: 0.6964 - val_loss: 0.5952
Epoch 7/30
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7075 - loss: 0.5730 - val_accuracy: 0.7036 - val_loss: 0.5913
Epoch 8/30
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7106 - loss: 0.5706 - val_accuracy: 0.7000 - val_loss: 0.5965
Epo

In [33]:
ns_model, ns_accuracy, x_ns_test, y_ns_test = train_model(x, df["NS"])

Epoch 1/30
79/79 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.5194 - loss: 0.6946 - val_accuracy: 0.5857 - val_loss: 0.6766
Epoch 2/30
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5737 - loss: 0.6850 - val_accuracy: 0.5857 - val_loss: 0.6690
Epoch 3/30
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5986 - loss: 0.6737 - val_accuracy: 0.5964 - val_loss: 0.6602
Epoch 4/30
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5801 - loss: 0.6710 - val_accuracy: 0.6143 - val_loss: 0.6613
Epoch 5/30
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6123 - loss: 0.6584 - val_accuracy: 0.5857 - val_loss: 0.6670
Epoch 6/30
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5978 - loss: 0.6611 - val_accuracy: 0.6107 - val_loss: 0.6624
Epoch 7/30
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6332 - loss: 0.6491 - val_accuracy: 0.6179 - val_loss: 0.6609
Epoch 8/30
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6138 - loss: 0.6589 - val_accuracy: 0.6071 - val_loss:

In [34]:
tf_model, tf_accuracy, x_tf_test, y_tf_test = train_model(x, df["TF"])

Epoch 1/30
79/79 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.6327 - loss: 0.6535 - val_accuracy: 0.6536 - val_loss: 0.6226
Epoch 2/30
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7159 - loss: 0.5807 - val_accuracy: 0.6821 - val_loss: 0.6205
Epoch 3/30
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7149 - loss: 0.5879 - val_accuracy: 0.6929 - val_loss: 0.6088
Epoch 4/30
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7206 - loss: 0.5700 - val_accuracy: 0.6857 - val_loss: 0.6100
Epoch 5/30
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7138 - loss: 0.5830 - val_accuracy: 0.7000 - val_loss: 0.6169
Epoch 6/30
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7291 - loss: 0.5680 - val_accuracy: 0.7000 - val_loss: 0.6046
Epoch 7/30
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7213 - loss: 0.5794 - val_accuracy: 0.6964 - val_loss: 0.6058
Epoch 8/30
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7091 - loss: 0.5834 - val_accuracy: 0.7036 - val_loss:

In [35]:
jp_model, jp_accuracy, x_jp_test, y_jp_test = train_model(x, df["JP"])

Epoch 1/30
79/79 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.5361 - loss: 0.6901 - val_accuracy: 0.6571 - val_loss: 0.6355
Epoch 2/30
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6008 - loss: 0.6661 - val_accuracy: 0.6750 - val_loss: 0.6365
Epoch 3/30
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6122 - loss: 0.6588 - val_accuracy: 0.6607 - val_loss: 0.6314
Epoch 4/30
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6210 - loss: 0.6517 - val_accuracy: 0.6893 - val_loss: 0.6312
Epoch 5/30
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6030 - loss: 0.6637 - val_accuracy: 0.6821 - val_loss: 0.6282
Epoch 6/30
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6216 - loss: 0.6487 - val_accuracy: 0.6571 - val_loss: 0.6347
Epoch 7/30
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6196 - loss: 0.6458 - val_accuracy: 0.6750 - val_loss: 0.6354
Epoch 8/30
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6295 - loss: 0.6405 - val_accuracy: 0.6607 - val_loss:

In [36]:
model_info = {
    "ie_model": (ie_model, ie_accuracy),
    "ns_model": (ns_model, ns_accuracy),
    "tf_model": (tf_model, tf_accuracy),
    "jp_model": (jp_model, jp_accuracy),
}

### SAVE MODELS

In [37]:
# load record
with open("./models/accuracy_record.json", "r") as f:
    config = json.load(f)
    
def new_save(model, accuracy, name):
    print(accuracy, config[name])
    if accuracy > config[name]:
        config[name] = accuracy
        model.save(f"./models/{name}.keras")  

for name, (model, accuracy) in model_info.items():
    new_save(model, accuracy, name)

# save record
with open("./models/accuracy_record.json", "w") as f:
    json.dump(config, f, indent=4)

0.7289407849311829 0
0.6230191588401794 0
0.7197664976119995 0
0.5963302850723267 0


### LOAD MODEL FROM SAVE

In [12]:
def load_from_save(name):
    return load_model(f"./models/{name}.keras")

# for name, (model, _) in model_info.items():
#     model = load_from_save(name)

ie_model = load_from_save("ie_model")
ns_model = load_from_save("ns_model")
tf_model = load_from_save("tf_model")
jp_model = load_from_save("jp_model")

### TEST PREDICTION

In [34]:
ie_pred = (ie_model.predict(x_ie_test) > 0.5).astype(int).reshape(-1)
ns_pred = (ns_model.predict(x_ns_test) > 0.5).astype(int).reshape(-1)
tf_pred = (tf_model.predict(x_tf_test) > 0.5).astype(int).reshape(-1)
jp_pred = (jp_model.predict(x_jp_test) > 0.5).astype(int).reshape(-1)

# Map to MBTI
def get_mbti(ie, ns, tf, jp):
    return f"{'E' if ie else 'I'}{'S' if ns else 'N'}{'F' if tf else 'T'}{'P' if jp else 'J'}"

pred_mbti = [get_mbti(ie_pred[i], ns_pred[i], tf_pred[i], jp_pred[i]) for i in range(len(ie_pred))]

actual_mbti = [
    get_mbti(y_ie_test.iloc[i], y_ns_test.iloc[i], y_tf_test.iloc[i], y_jp_test.iloc[i])
    for i in range(len(y_ie_test))
]

# So sánh
for p, a in zip(pred_mbti[20:], actual_mbti[20:]):
    print(f"Pred: {p:<10} | Actual: {a}")


ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense_180" is incompatible with the layer: expected axis -1 of input shape to have value 10, but received input with shape (32, 8)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(32, 8), dtype=float32)
  • training=False
  • mask=None
  • kwargs=<class 'inspect._empty'>

### MORE TEST

In [32]:
mbti = "INTJ"

test_df = pd.read_csv(f"./csv/{mbti}_df.csv")

test_data = test_df[playlist_features]

ie_pred = (ie_model.predict(test_data) > 0.5).astype(int).reshape(-1)
ns_pred = (ns_model.predict(test_data) > 0.5).astype(int).reshape(-1)
tf_pred = (tf_model.predict(test_data) > 0.5).astype(int).reshape(-1)
jp_pred = (jp_model.predict(test_data) > 0.5).astype(int).reshape(-1)

# Map to MBTI
def get_mbti(ie, ns, tf, jp):
    return f"{'E' if ie else 'I'}{'S' if ns else 'N'}{'F' if tf else 'T'}{'P' if jp else 'J'}"

res = []

for i in range(len(ie_pred)):
    mbti_pred = get_mbti(ie_pred[i], ns_pred[i], tf_pred[i], jp_pred[i])
    res.append(mbti_pred)

table = pd.DataFrame(res).drop_duplicates()
table


ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense_180" is incompatible with the layer: expected axis -1 of input shape to have value 10, but received input with shape (32, 8)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(32, 8), dtype=float32)
  • training=False
  • mask=None
  • kwargs=<class 'inspect._empty'>

### TRAIN ACCORDING TO FUNCTION PAIR

In [38]:
x = df[playlist_features]
# print(x.corr())
x = scaler.fit_transform(x)
dump(scaler, "models/func_pair_scaler.pkl") 

encoder = LabelEncoder()
y = encoder.fit_transform(df["function_pair"])
y

array([0, 0, 0, ..., 2, 2, 2])

In [39]:
x_test, x_train, y_test, y_train = train_test_split(
    x, y,
    test_size=0.2,
    random_state=42,
    shuffle = True)

In [40]:
func_pair_model = Sequential([
    keras.layers.Dense(256, activation="relu", input_dim=x_train.shape[1]),
    keras.layers.Dropout(0.3),

    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dropout(0.3),

    keras.layers.Dense(64, activation="relu"),
    keras.layers.Dropout(0.2),

    keras.layers.Dense(32, activation="relu"),
    keras.layers.Dropout(0.1),

    keras.layers.Dense(4, activation="softmax")  
])

C:\Users\DELL\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [41]:
func_pair_model.compile(
        optimizer= "adam",
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'],
    )

In [42]:
func_pair_model.fit(
        x_train, y_train,
        validation_split = 0.1,
        epochs = 30,
        verbose = 1)

loss, func_pair_accuracy = func_pair_model.evaluate(x_test, y_test, verbose=0)

Epoch 1/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.2568 - loss: 1.3943 - val_accuracy: 0.4250 - val_loss: 1.3747
Epoch 2/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3634 - loss: 1.3423 - val_accuracy: 0.4000 - val_loss: 1.3607
Epoch 3/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4350 - loss: 1.2778 - val_accuracy: 0.4125 - val_loss: 1.3487
Epoch 4/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4554 - loss: 1.2618 - val_accuracy: 0.4000 - val_loss: 1.2662
Epoch 5/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4524 - loss: 1.2570 - val_accuracy: 0.4500 - val_loss: 1.2696
Epoch 6/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5148 - loss: 1.2124 - val_accuracy: 0.4375 - val_loss: 1.2656
Epoch 7/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4365 - loss: 1.2650 - val_accuracy: 0.4125 - val_loss: 1.2503
Epoch 8/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4683 - loss: 1.2183 - val_accuracy: 0.4375 - val_loss

In [44]:
with open("./models/accuracy_record.json") as f:
    config = json.load(f)
    print(func_pair_accuracy, config["func_pair_model"])
    
    if func_pair_accuracy > config.get("func_pair_model", 0):
        config["func_pair_model"] = func_pair_accuracy
        func_pair_model.save("./models/func_pair_model.keras")

with open("./models/accuracy_record.json", "w") as f:
    json.dump(config, f, indent=4)

0.44976526498794556 0.44976526498794556
